In [54]:
#install wikipedia
!pip install wikipedia

In [55]:
#install google search
!pip install Google-Search-API

In [56]:
#import libraries
import unicodedata
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk, conlltags2tree, tree2conlltags
from google import google
import wikipedia
import collections

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
# Determine the type of question whether it is location, date, person or definiton
def classify_question(question):
  q = question.lower().split()
  if q[0] == 'where':
    questionType = 'Location'
  elif 'year'  in question:
    questionType = 'Date'
  elif 'country' in question:
    questionType = 'Country'
  elif q[0] == 'who':
    questionType = 'Person'
  elif q[0] == 'what':
    questionType = 'Definition'
  else:
    questionType = 'None'
  return questionType

In [0]:
#google search
def google_search(question):
  first_page = google.search(question,1)
  #print first_page
  top_three_result = []
  i = 0
  while i<5:
    top_three_result.append(first_page[i].description)
    i+=1

  first_search = ''.join(top_three_result)
  #print first_search

  ne_tree = (ne_chunk(pos_tag(word_tokenize(first_search))))

  iob_tagged = tree2conlltags(ne_tree)

  ss = [tuple(map(str,eachTuple)) for eachTuple in iob_tagged]
  question_type = classify_question(question)
  print('question_type: ', question_type)
  if question_type == 'None':
      ans = "Oops! I don't know."
  else:
      google_answer = []
      if question_type == 'Person':
          for i in range(len(ss)):
              if ss[i][2] == 'B-PERSON'or ss[i][2] == 'I-PERSON':
                  google_answer.append(ss[i][0])
      elif question_type == 'Country':
          print('country identified')
          for i in range(len(ss)):
              if ss[i][2] == 'B-GPE'or ss[i][2] == 'I-GPE':
                  google_answer.append(ss[i][0])
      elif question_type == 'Location':
          for i in range(len(ss)):
              if ss[i][2] == 'B-LOCATION'or ss[i][2] == 'I-LOCATION':
                  google_answer.append(ss[i][0])
      elif question_type == 'Date':
          for i in range(len(ss)):
              if ss[i][2] == 'B-DATE'or ss[i][2] == 'I-DATE':
                  google_answer.append(ss[i][0])
      print('google: ',google_answer)
      if not google_answer:
          ans = "Oops, I don't know! "
      else:
          print('inside else')
          counts = collections.Counter(google_answer)
          print('counts: ',counts)
          t = counts.most_common(4)
          candidate_answer =  [ seq[0] for seq in t ]
          print(candidate_answer)
          for i in range(len(candidate_answer)):
              candidate_answer[i] = 'Candidate Answer '+ str(i+1)+' '+ candidate_answer[i]
          candidate_answer = '\n'.join(candidate_answer)
          ans = candidate_answer
  return ans


In [0]:
#wikipedia search
def wiki_search(question):
  l = question.split(' ')
  if len(l) > 2:
    ques = " ".join(l[2:])
  try:
    print('inside wiki search')
    ans = (wikipedia.summary(question, sentences=1))
    link = wikipedia.page(ques)
    ans = ans + '\n For more information: '+link.url
  except:
    print('wiki_search_failed_google')
    google_search(question)
  return ans


In [0]:
def answer_question(question):
  q_type = classify_question(question)
  print(q_type)
  if q_type == 'Definition' or q_type == 'Location':
    print('except-wiki')
    ans = wiki_search(question)
  #if len(question.split())<=5:
  #    print 'none-wiki'
  #    ans = wiki_search(question)
  else:
    ans = google_search(question)
    print('google answ: ',ans)
  return ans


In [61]:
print(answer_question('Who is the president of United States?'))
# print(answer_question('Who is the president of Iran?'))
#print(answer_question('Where is Tehran?'))
#print(answer_question('What is the name of Canada\'s capital city?'))
#print(answer_question('Who directed Gone with the wind?'))

Person
question_type:  Person
google:  ['Donald', 'Donald', 'Trump', 'Social', 'Donald', 'J.', 'Trump', 'Hold', 'Office']
inside else
counts:  Counter({'Donald': 3, 'Trump': 2, 'Social': 1, 'J.': 1, 'Hold': 1, 'Office': 1})
['Donald', 'Trump', 'Social', 'J.']
google answ:  Candidate Answer 1 Donald
Candidate Answer 2 Trump
Candidate Answer 3 Social
Candidate Answer 4 J.
Candidate Answer 1 Donald
Candidate Answer 2 Trump
Candidate Answer 3 Social
Candidate Answer 4 J.
